In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON


In [2]:
def get_entity_for_musician(artist):
    # YAGO SPARQL endpoint
    endpoint_url = "https://yago-knowledge.org/sparql/query"

    # Randomly select an artist from the list
    artist = artist.lower()

    # SPARQL query template
    query_template = f"""
    PREFIX yago: <http://yago-knowledge.org/resource/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX schema: <http://schema.org/>

    SELECT DISTINCT ?entity ?type ?influencedBy WHERE {{
      {{
        ?entity a ?type ;
                rdfs:label ?label .
        FILTER(CONTAINS(LCASE(STR(?label)), \"{artist}\")).
        ?type rdfs:subClassOf* schema:MusicGroup .
      }}
      UNION
      {{
        ?entity a yago:Musician ;
                rdfs:label ?label .
        FILTER(CONTAINS(LCASE(STR(?label)), \"{artist}\")).
      }}
      UNION      
      {{
        ?entity a ?type ;
                rdfs:label ?label .
        FILTER(CONTAINS(LCASE(STR(?label)), \"{artist}\")).
        ?type rdfs:subClassOf* yago:Musician .
      }}
      OPTIONAL {{
        ?entity schema:influencedBy ?influencedBy .
      }}
    }} LIMIT 10
    """

    # Setup SPARQL
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query_template)
    sparql.setReturnFormat(JSON)

    try:
        # Execute the query
        results = sparql.query().convert()
        bindings = results["results"]["bindings"]
        results = []
        for binding in  bindings:
            entity = binding["entity"]["value"]
            #label = binding.get("label", {}).get("value", "Unknown Label")
            type_ = binding.get("type", {}).get("value", "Unknown Type")
            influenced_by = binding.get("influencedBy", {}).get("value", None)
            results.append( {
                "artist": artist,
                "entity": entity,
               # "label": label,
                "type": type_,
                "influenced_by": influenced_by
            })
        return results

    except Exception as e:
        return None

In [3]:
def get_songs_by_artist_url(artist_url):
    # YAGO SPARQL endpoint
    endpoint_url = "https://yago-knowledge.org/sparql/query"

    # SPARQL query template
    query_template = f"""
    PREFIX schema: <http://schema.org/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT DISTINCT ?entity (SAMPLE(?label) AS ?label) ?type WHERE {{
      ?entity a ?type ;
              schema:musicBy <{artist_url}> ;
              rdfs:label ?label .
      ?type rdfs:subClassOf* schema:MusicComposition .
      FILTER(LANG(?label) = "en")
    }} GROUP BY ?entity ?type LIMIT 50
    """

    # Setup SPARQL
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query_template)
    sparql.setReturnFormat(JSON)

    try:
        # Execute the query
        results = sparql.query().convert()
        bindings = results["results"]["bindings"]

        songs = []
        for binding in bindings:
            entity = binding.get("entity", {}).get("value", "Unknown Entity")
            label = binding.get("label", {}).get("value", "Unknown Label")
            type_ = binding.get("type", {}).get("value", "Unknown Type")
            songs.append({"entity": entity, "label": label, "type": type_})

        return songs

    except Exception as e:
        print(str(e))
        return None

In [4]:

def get_label_by_entity_url(entity_url):
    # YAGO SPARQL endpoint
    endpoint_url = "https://yago-knowledge.org/sparql/query"

    # SPARQL query template
    query_template = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?label WHERE {{
      <{entity_url}> rdfs:label ?label .
      
      FILTER(LANG(?label) = "en")
    }} LIMIT 1
    """

    # Setup SPARQL
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query_template)
    sparql.setReturnFormat(JSON)

    try:
        # Execute the query
        results = sparql.query().convert()
        bindings = results["results"]["bindings"]

        if bindings:
            sampled_label = bindings[0].get("label", {}).get("value", None)
            return {"entity": entity_url, "label": sampled_label}
        else:
            return {"entity": entity_url, "label": "No label found."}

    except Exception as e:
        return {"error": str(e)}

In [5]:
#get_label_by_entity_url("http://yago-knowledge.org/resource/David_Bowie")

In [6]:
#get_songs_by_artist_url("http://yago-knowledge.org/resource/David_Bowie")

In [7]:
import random
import numpy as np
def get_recommendations_based_on_influencedBy(artist_str, max_influences = 2):
    results = []
    print("Getting influencedBy entities...")
    getting_items = get_entity_for_musician(artist_str)
    print(getting_items)
    influences = np.unique(list(map(lambda item: item["influenced_by"], [item for item in getting_items if item["influenced_by"] is not None]))).tolist()

    if len(influences) == 0:
        return None
    print(influences)
    influences = random.choices(influences, k=min(max_influences, len(influences)))
    
    print(f"Choosing first {max_influences} artists...")
    for influence in influences:
        print(f"Processing {influence}...")
        dict_item = get_label_by_entity_url(influence)
        songs = get_songs_by_artist_url(influence)
        results.append({"artist":influence, "label":dict_item["label"], "songs":songs})
        
    return results



In [8]:

def get_randomly_weighted(items, k):
    artists = list(items.keys())
    weights = list(items.values())
    selected = {}

    for _ in range(k):
        total_weight = sum(weights)
        probabilities = [w / total_weight for w in weights]

        chosen_index = random.choices(range(len(artists)), probabilities, k=1)[0]
        selected[artists[chosen_index]] =  weights[chosen_index]

        artists.pop(chosen_index)
        weights.pop(chosen_index)

    return selected
    
def get_recommendarions_based_on_influencedBy_likes_dislikes(likes = [], dislikes = [], max_influences=4, debug=False):
    influencedByCounters  = {}
    results = []
    for artist in likes:
        if debug:
            print(f"Processing liked artist: {artist} ")    
        getting_items = get_entity_for_musician(artist)
        for item in getting_items:
            if item["influenced_by"] is not None:
                if item["influenced_by"] not in influencedByCounters:
                    influencedByCounters[item["influenced_by"]] = 1
                else:
                    influencedByCounters[item["influenced_by"]] += 1
    for artist in dislikes:
        if debug:
            print(f"Processing disliked artist: {artist} ")   
        
        getting_items = get_entity_for_musician(artist)
        for item in getting_items:
            if item["influenced_by"] is not None:
                if item["influenced_by"] in influencedByCounters:
                    influencedByCounters[item["influenced_by"]] -= 1
                    
                    if debug:
                        print(f'{item["influenced_by"]} was penalized')
                    if influencedByCounters[item["influenced_by"]] == 0:
                        del influencedByCounters[item["influenced_by"]]
    if len(influencedByCounters) == 0:
        return None
    
    influenced_by_dict = get_randomly_weighted(influencedByCounters,max_influences)    
    for influence, count in influenced_by_dict.items():
        if debug:
            print(f"Processing {influence}...")
        dict_item = get_label_by_entity_url(influence)
        songs = get_songs_by_artist_url(influence)
        results.append({"artist":influence, "label":dict_item["label"], "songs":songs, "weight":count})
    return sorted(results, key=lambda item: item["weight"], reverse=True)

In [9]:
likes = ["beatles", "Bruno Mars", "Michael Jackson", "Britney Spears"]
dislikes = ["Sam Smith"]
songs = get_recommendarions_based_on_influencedBy_likes_dislikes(likes, dislikes, max_influences=5, debug=True)

Processing liked artist: beatles 
Processing liked artist: Bruno Mars 
Processing liked artist: Michael Jackson 
Processing liked artist: Britney Spears 
Processing disliked artist: Sam Smith 
http://yago-knowledge.org/resource/Whitney_Houston was penalized
Processing http://yago-knowledge.org/resource/Charlie_Chaplin...
Processing http://yago-knowledge.org/resource/Marcel_Marceau...
Processing http://yago-knowledge.org/resource/Diana_Ross...
Processing http://yago-knowledge.org/resource/Michael_Jackson...
Processing http://yago-knowledge.org/resource/James_Brown...


In [10]:
songs

[{'artist': 'http://yago-knowledge.org/resource/Michael_Jackson',
  'label': 'Michael Jackson',
  'songs': [{'entity': 'http://yago-knowledge.org/resource/The_Way_You_Make_Me_Feel',
    'label': 'The Way You Make Me Feel',
    'type': 'http://yago-knowledge.org/resource/Musical_Work_U002F_Composition_Q105543609'},
   {'entity': 'http://yago-knowledge.org/resource/Give_In_to_Me',
    'label': 'Give In to Me',
    'type': 'http://yago-knowledge.org/resource/Musical_Work_U002F_Composition_Q105543609'},
   {'entity': 'http://yago-knowledge.org/resource/Leave_Me_Alone__u0028_Michael_Jackson_song_u0029_',
    'label': 'Leave Me Alone',
    'type': 'http://yago-knowledge.org/resource/Musical_Work_U002F_Composition_Q105543609'},
   {'entity': 'http://yago-knowledge.org/resource/Can_You_Feel_It__u0028_The_Jacksons_song_u0029_',
    'label': 'Can You Feel It',
    'type': 'http://yago-knowledge.org/resource/Musical_Work_U002F_Composition_Q105543609'},
   {'entity': 'http://yago-knowledge.org/res